In [1]:
################################################
### mit dem visualize2 envirionment ###########
#### weil plotly 4.0!!#########################
###############################################

import pandas as pd
import numpy as np
import sqlite3

import re
import spacy
nlp = spacy.load("de_core_news_sm")


import plotly.offline as py # wichtig hier als .offline importieren. 
import plotly.figure_factory as ff
import plotly.graph_objs as go
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
py.offline.init_notebook_mode(connected=True)
init_notebook_mode(connected=True)



In [2]:
con = sqlite3.connect("C:/Users/tschu/OneDrive/Master Data Science/1_Master Thesis/million_post_corpus/corpus.sqlite3")
txt = pd.read_sql("SELECT * FROM Posts ", con)
annos = pd.read_sql("SELECT * from Annotations_consolidated", con)

In [3]:
def dataPrep(posts):
    
    df = pd.merge(txt, posts, on="ID_Post")
    
    df = pd.DataFrame({
    'ID_Post':df["ID_Post"],
    'label':df["Value"],
    "headline": df["Headline"],
    'alpha':['a']*df.shape[0],
    'text': df["Body"]})
    
    df= df.fillna("_biliq_")
    
    df["kombi"] = df.headline + " " + df.text
    
    df["kombi"] = df["kombi"].str.replace("_biliq_", " ")
    
    df["kombi"] = df["kombi"].replace(r'http\S+',"", regex=True)
    
    
    
    df = df.replace('  ', np.nan)
     
    #nancheck = df[df.isna().any(axis=1)]
    
    #print(nancheck.label.value_counts())
    
    df=df.dropna(subset=["kombi"])
    
    df["kombi"] = df["kombi"].replace("[^a-zA-Z\Ä\ä\Ü\ü\Ö\ö\ß\?\!\.]+", " ", regex=True)
    
    
    df = pd.DataFrame({
    'id':range(len(df)),
    'label':df["label"],
    'alpha':['a']*df.shape[0],
    'text': df["kombi"]})
    
    df["doc"] = [nlp(text) for text in df.text]
    df["number_of_tokens"] = [len(token) for token in df.doc]
    
    print("dataprep finished")
    
    return df
  


In [26]:
def bringTSV(train, dev):
    
    train.to_csv("data/train.tsv", sep="\t", index = False, header=False)
    print("wrote train to file.....")
    dev.to_csv("data/dev.tsv", sep="\t", index = False, header=False)
    print("wrote dev to file....")


In [4]:
argument0 = annos[annos["Category"].isin(['ArgumentsUsed'])]
diskrim0 = annos[annos["Category"].isin(['Discriminating'])]
inapp0 = annos[annos["Category"].isin(['Inappropriate'])]
offto0 = annos[annos["Category"].isin(['OffTopic'])]
seneu0 = annos[annos["Category"].isin(['SentimentNeutral'])]
seneg0 = annos[annos["Category"].isin(['SentimentNegative'])]
sepos0 = annos[annos["Category"].isin(['SentimentPositive'])]
perso0 = annos[annos["Category"].isin(["PersonalStories"])]
feedb0 = annos[annos["Category"].isin(["PossiblyFeedback"])]

In [5]:
Kann_ni_gelesen_werden_ar = dataPrep(argument0)
diskrim = dataPrep(diskrim0)
inapp= dataPrep(inapp0)
offto=dataPrep(offto0)
seneu=dataPrep(seneu0)
seneg=dataPrep(seneg0)
sepos=dataPrep(sepos0)
perso=dataPrep(perso0)
Kann_ni_gelesen_werden_fb=dataPrep(feedb0)


dataprep finished
dataprep finished
dataprep finished
dataprep finished
dataprep finished
dataprep finished
dataprep finished
dataprep finished
dataprep finished


In [16]:
x0= diskrim.number_of_tokens
x1= Kann_ni_gelesen_werden_fb.number_of_tokens
x2= perso.number_of_tokens

fig = go.Figure()
fig.add_trace(go.Histogram(x=x0, name="restliche Katgorien"))
fig.add_trace(go.Histogram(x=x1, name="Feedback"))
fig.add_trace(go.Histogram(x=x2, name="Persönliche Geschichte"))

# The two histograms are drawn on top of another
fig.layout.update(barmode='stack',
                  #title="Verteilung der Tokenlänge",
                  xaxis_title="Anzahl der Token pro Post",
                  yaxis_title="Anzahl der Posts"
                 )
fig.show()
#fig.iplot()

## Ausreisser bereiningen

In [17]:
Kann_ni_gelesen_werden_ar_corr = Kann_ni_gelesen_werden_ar[(Kann_ni_gelesen_werden_ar["number_of_tokens"] >=3) & (Kann_ni_gelesen_werden_ar["number_of_tokens"] <=160)]
diskrim_corr = diskrim[(diskrim["number_of_tokens"] >=3) & (diskrim["number_of_tokens"] <=160)]
inapp_corr = inapp[(inapp["number_of_tokens"] >=3) & (inapp["number_of_tokens"] <=160)]
offto_corr = offto[(offto["number_of_tokens"] >=3) & (offto["number_of_tokens"] <=160)]
seneu_corr = seneu[(seneu["number_of_tokens"] >=3) & (seneu["number_of_tokens"] <=160)]
seneg_corr = seneg[(seneg["number_of_tokens"] >=3) & (seneg["number_of_tokens"] <=160)]
sepos_corr = sepos[(sepos["number_of_tokens"] >=3) & (sepos["number_of_tokens"] <=160)]
perso_corr = perso[(perso["number_of_tokens"] >=3) & (perso["number_of_tokens"] <=160)]
Kann_ni_gelesen_werden_fb_corr = Kann_ni_gelesen_werden_fb[(Kann_ni_gelesen_werden_fb["number_of_tokens"] >=3) & (Kann_ni_gelesen_werden_fb["number_of_tokens"] <=160)]




In [28]:
x0= diskrim_corr.number_of_tokens
x1= Kann_ni_gelesen_werden_fb_corr.number_of_tokens
x2= perso_corr.number_of_tokens

fig = go.Figure()
fig.add_trace(go.Histogram(x=x0, name="restliche Kategorien",  marker_color = 'blue'))
fig.add_trace(go.Histogram(x=x1, name="Feedback",marker_color = 'red'))
fig.add_trace(go.Histogram(x=x2, name="Persönliche Geschichte",marker_color = 'green'))

# The two histograms are drawn on top of another
fig.layout.update(barmode='stack',
                  #title="Länge Posts (Ausreißer bereinigt)",
                  xaxis_title="Anzahl der Token pro Post",
                  yaxis_title="Anzahl der Posts"
                 )
fig.show()
#fig.iplot()

In [29]:
y0= perso_corr.number_of_tokens
y1= Kann_ni_gelesen_werden_fb_corr.number_of_tokens
y2= diskrim_corr.number_of_tokens

fig = go.Figure()
fig.add_trace(go.Box(y=y0, marker_color = 'green', name="Persönliche Geschichte"))
fig.add_trace(go.Box(y=y1, marker_color = 'red', name="Feedback" ))
fig.add_trace(go.Box(y=y2, marker_color = 'blue', name="restliche Kategorien"))

fig.layout.update(
                  #title="Verteilung der Tokenlänge (Ausreißer bereinigt)",

                  yaxis_title="Anzahl der Posts"
                 )


fig.show()

In [22]:
df_dict = {"Kann_ni_gelesen_werden_ar_corr" :Kann_ni_gelesen_werden_ar_corr, "diskrim_corr":diskrim_corr, "inapp_corr":inapp_corr, "offto_corr":offto_corr , "seneu_corr":seneu_corr,
         "seneg_corr": seneg_corr, "sepos_corr" :sepos_corr, "perso_corr": perso_corr, "Kann_ni_gelesen_werden_fb_corr":Kann_ni_gelesen_werden_fb_corr}


In [23]:
for k,v in df_dict.items():
    print(k)
    print(len(v.label))
    print(len(v[v['label'] == 1]))
    print(len(v[v['label'] == 0]))
    

Kann_ni_gelesen_werden_ar_corr
3550
1013
2537
diskrim_corr
3550
281
3269
inapp_corr
3550
299
3251
offto_corr
3550
570
2980
seneu_corr
3550
1830
1720
seneg_corr
3550
1677
1873
sepos_corr
3550
43
3507
perso_corr
9194
1625
7569
Kann_ni_gelesen_werden_fb_corr
5968
1300
4668


In [30]:
# vor Datenbereinigung

#categories = ["Persönliche Geschichte", "Feedback","Argument","Sentiment negativ","Sentiment neutral","Sentiment positiv","Off-Topic","Unangemessen","Diskriminierend"]
#y1=[1625,1301,1022,1691,1865,43,580,303,282]
#y0=[7711,4737,2577,1908,1734,3556,3019,3296,3317]

# nach Datenbereinigung 
#categories = ["Persönliche Geschichte", "Feedback","Argument","Sentiment negativ","Sentiment neutral","Sentiment positiv","Off-Topic","Unangemessen","Diskriminierend"]
#y1=[1625,1301,1015,1689,1851,43,577,303,282]
#y0 = [7689,4714, 2568, 1894, 1732, 3540, 3006, 3280, 3301]

# Nach der bereinigten Bereinigung:
categories = ["Argument", "Diskriminierend", "Unangemessen", "Off-Topic","Sentiment neutral", "Sentiment negativ","Sentiment positiv","Persönliche Geschichte","Feedback"]
y1=[1013,281,299,570, 1830, 1677, 43, 1625,1300]
y0=[2537,3269,3251,2980, 1720, 1873, 3507,7569, 4668]


fig = go.Figure(data=[
    go.Bar(name="ja", x=categories, y=y1, text = y1, textposition="outside",  marker=dict(
        color='lime')),
    go.Bar(name="nein", x= categories, y=y0, text=y0, textposition="outside", marker=dict(color="lightseagreen"))
])

fig.layout.update(
    barmode="group",
    title = "",
    yaxis=dict(
    title= "Anzahl der Posts")
)

fig.show()
      

In [31]:
#Export the Data with a loop and get rid of the unneeded columns:


for k, v in df_dict.items():
    v = v.drop(columns=['doc', 'number_of_tokens'])
    v.to_csv("2019-12-02-clean_files_for_Aug/"+k+".tsv", sep="\t", index = False, header=True)
    

In [69]:
#check ob das ladnen funkt:
#testoladdiskrim = pd.read_csv("2019_11_25_Data_Augmenation_Data\diskrim.tsv", sep="\t", header = None)

### Probleme beim Einlesen für nächsten Schritt:
FileNotFoundError: File b'2019-12-02-clean_files_for_Aug\x0ceedb_corr.tsv' does not exist
FileNotFoundError: File b'2019-12-02-clean_files_for_Aug\x07rgument_corr.tsv' does not exist
beide können nicht eingelesen werden. 

werden die beiden Daten beim Exprot nach der Datenbereingung in 
Kann_ni_gelesen_werden_fb_corr
und 
Kann_ni_gelesen_werden_ar_corr
umbenannt, funktioniert das Einlesen.